<a href="https://colab.research.google.com/drive/1DBiEoUHc7YftJdDkbJaEc3-BOE5BCwNR" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





In [1]:
!apt-get install -y swig cmake ffmpeg
!pip uninstall -y box2d-py
!pip install gymnasium[box2d] pygame

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (3,258 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb 

In [ ]:
# ✅ Colab Notebook Setup for Reinforcement Learning (DQN, PPO) with GPU Support

# Install required dependencies
!pip install stable-baselines3 torch pandas numpy requests matplotlib seaborn -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import requests
import io

# ✅ Load Metadata (Building Characteristics)
METADATA_URL = "https://media.githubusercontent.com/media/buds-lab/building-data-genome-project-2/refs/heads/master/data/metadata/metadata.csv"
response = requests.get(METADATA_URL)
metadata_df = pd.read_csv(io.BytesIO(response.content))

# ✅ Load Energy Dataset
ENERGY_DATA_URL = "https://github.com/buds-lab/building-data-genome-project-2/raw/master/data/meters/kaggle/kaggle.csv"
response = requests.get(ENERGY_DATA_URL)
energy_df = pd.read_csv(io.BytesIO(response.content))

# ✅ Convert Metadata 'building_id' to match Energy Dataset
metadata_mapping = {name: idx for idx, name in enumerate(metadata_df["building_id"].unique())}
metadata_df["building_id"] = metadata_df["building_id"].map(metadata_mapping)
energy_df["building_id"] = energy_df["building_id"].astype(int)  # Ensure integer format

# ✅ Merge Energy Data with Metadata
df = energy_df.merge(metadata_df, on="building_id", how="left")

# ✅ Select Relevant Features
df = df[["building_id", "meter", "meter_reading", "sqm", "numberoffloors", "yearbuilt", "eui"]]

# ✅ Fill Missing Values
df = df.fillna(0)

In [6]:
energy_df["building_id"] = energy_df["building_id"].astype(int)  # Ensure integer format
metadata_df['building_id'].max()

1635

In [8]:
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ndcg_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import warnings

warnings.filterwarnings("ignore")

# Download NLTK dependencies
nltk.download("stopwords")
nltk.download("punkt")

# -------------------------------
# Step 1: Load MovieLens 1M Dataset (Larger Dataset)
# -------------------------------
MOVIELENS_URL = "https://files.grouplens.org/datasets/movielens/ml-1m.zip"

# Download the ZIP file in-memory
response = requests.get(MOVIELENS_URL)
zip_file = zipfile.ZipFile(BytesIO(response.content))

# Read ratings.dat and movies.dat
with zip_file.open("ml-1m/ratings.dat") as file:
    ratings = pd.read_csv(file, sep="::", names=["userId", "movieId", "rating", "timestamp"], engine="python", encoding="ISO-8859-1")

with zip_file.open("ml-1m/movies.dat") as file:
    movies = pd.read_csv(file, sep="::", names=["movieId", "title", "genres"], engine="python", encoding="ISO-8859-1")

# Merge Ratings & Movies
df = ratings.merge(movies, on="movieId")

# Convert userId & movieId to categorical codes
df["userId"] = df["userId"].astype("category").cat.codes
df["movieId"] = df["movieId"].astype("category").cat.codes

# -------------------------------
# Step 2: Simulate User Click Data
# -------------------------------
df["click"] = (df["rating"] >= 4).astype(int)

# -------------------------------
# Step 3: Multi-Armed Bandit (MAB) with Upper Confidence Bound (UCB1)
# -------------------------------
class MultiArmedBandit:
    def __init__(self, movie_ids):
        self.movie_rewards = {movie_id: 0 for movie_id in movie_ids}
        self.movie_attempts = {movie_id: 1 for movie_id in movie_ids}

    def select_movie(self, num_recs=5):
        selected_movies = sorted(
            self.movie_rewards.keys(), 
            key=lambda movie: self.movie_rewards[movie] / self.movie_attempts[movie] + np.sqrt(2 * np.log(sum(self.movie_attempts.values())) / self.movie_attempts[movie]), 
            reverse=True
        )
        return selected_movies[:num_recs]

    def update(self, movie_id, reward):
        self.movie_attempts[movie_id] += 1
        self.movie_rewards[movie_id] += reward

mab = MultiArmedBandit(df["movieId"].unique())

# -------------------------------
# Step 4: Markov Decision Process (MDP) with More State Transitions
# -------------------------------
class MarkovDecisionProcess:
    def __init__(self):
        self.transitions = {}
        self.rewards = {}

    def update(self, current_movie, next_movie, reward):
        key = (current_movie, next_movie)
        if key not in self.transitions:
            self.transitions[key] = 1
            self.rewards[key] = reward
        else:
            self.transitions[key] += 1
            self.rewards[key] = 0.9 * self.rewards[key] + 0.1 * reward

    def recommend_next(self, current_movie, num_recs=5):
        candidates = sorted(
            [(key[1], self.rewards[key]) for key in self.rewards if key[0] == current_movie], 
            key=lambda x: x[1], reverse=True
        )
        return [movie for movie, _ in candidates[:num_recs]] if candidates else random.choices(df["movieId"].unique(), k=num_recs)

mdp = MarkovDecisionProcess()

# -------------------------------
# Step 5: Q-Learning with Enhanced Learning Rate & Discount Factor
# -------------------------------
class QLearningRecommender:
    def __init__(self, num_movies, alpha=0.2, gamma=0.95, epsilon=0.2):
        self.num_movies = num_movies
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def update(self, state, action, reward, next_state):
        key = (state, action)
        if key not in self.q_table:
            self.q_table[key] = 0
        next_max = max([self.q_table.get((next_state, a), 0) for a in range(self.num_movies)], default=0)
        self.q_table[key] = (1 - self.alpha) * self.q_table[key] + self.alpha * (reward + self.gamma * next_max)

    def select_movie(self, state, num_recs=5):
        return [
            random.choice(range(self.num_movies))
            if random.random() < self.epsilon
            else max(range(self.num_movies), key=lambda x: self.q_table.get((state, x), 0))
            for _ in range(num_recs)
        ]

q_learning = QLearningRecommender(df["movieId"].nunique())

# -------------------------------
# Compare Accuracy & NDCG Score of Methods
# -------------------------------
def evaluate_recommendations(recommend_func, test_users, actual_clicks):
    predictions = []
    for user in test_users:
        recommendations = recommend_func()
        predictions.append(1 if any(movie in actual_clicks.get(user, []) for movie in recommendations) else 0)
    return predictions

test_users = df["userId"].unique()[:100]
actual_clicks = {user: df[(df["userId"] == user) & (df["click"] == 1)]["movieId"].tolist() for user in test_users}

mab_preds = evaluate_recommendations(mab.select_movie, test_users, actual_clicks)
mdp_preds = evaluate_recommendations(lambda: mdp.recommend_next(random.choice(df["movieId"].unique()), 5), test_users, actual_clicks)
ql_preds = evaluate_recommendations(lambda: q_learning.select_movie(random.choice(df["movieId"].unique()), 5), test_users, actual_clicks)

y_true = [1 if actual_clicks.get(user, []) else 0 for user in test_users]

def print_metrics(name, y_true, y_pred):
    print(f"{name} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{name} Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"{name} Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"{name} F1 Score: {f1_score(y_true, y_pred):.4f}")
    print(f"{name} NDCG Score: {ndcg_score([y_true], [y_pred]):.4f}\n")

print_metrics("MAB", y_true, mab_preds)
print_metrics("MDP", y_true, mdp_preds)
print_metrics("Q-Learning", y_true, ql_preds)


[nltk_data] Downloading package stopwords to /Users/ryoji/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ryoji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


MAB Accuracy: 0.9500
MAB Precision: 1.0000
MAB Recall: 0.9500
MAB F1 Score: 0.9744
MAB NDCG Score: 1.0000

MDP Accuracy: 0.1500
MDP Precision: 1.0000
MDP Recall: 0.1500
MDP F1 Score: 0.2609
MDP NDCG Score: 1.0000

Q-Learning Accuracy: 0.2900
Q-Learning Precision: 1.0000
Q-Learning Recall: 0.2900
Q-Learning F1 Score: 0.4496
Q-Learning NDCG Score: 1.0000

